Use this utlity to update the returns and std_dev fields within investment-options.csv

In [6]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [7]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [8]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [9]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [10]:
# read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# remove the exchange from the beginning of the symbol
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()
dow30['Symbol'] = dow30.apply(_symbol, axis=1)

dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M Company,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Pharmaceutical industry
AAPL,Apple Inc.,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar Inc.,Construction and Mining
CVX,Chevron Corporation,Petroleum industry
CSCO,Cisco Systems,Information technology
KO,The Coca-Cola Company,Food industry


In [11]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and Mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services'}

In [12]:
# map dow30 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
dow30.index = dow30.index.str.replace('.', '-')
dow30

,Description,Asset Class
Symbol,,
MMM,3M Company,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AAPL,Apple Inc.,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar Inc.,US Stocks:Industrials
CVX,Chevron Corporation,US Stocks:Energy
CSCO,Cisco Systems,US Stocks:Technology
KO,The Coca-Cola Company,US Stocks:Consumer Staples


In [13]:
# make symbols list
symbols = list(dow30.index)
#symbols

In [14]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

..............................


In [15]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DOW,...,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT,DIS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,137.07,84.23,135.28,24.86,113.66,75.83,85.62,22.78,34.62,nan,...,44.35,74.93,59.24,91.52,91.65,35.48,63.51,64.63,74.56,87.38
2015-01-05,133.98,82.01,133.68,24.16,112.87,71.83,82.20,22.33,34.62,nan,...,43.63,74.57,58.17,90.41,90.14,35.18,62.11,63.36,74.34,86.10
2015-01-06,132.55,80.26,129.37,24.16,111.54,71.37,82.16,22.32,34.88,nan,...,43.38,74.23,57.20,89.61,89.96,35.54,61.71,63.52,74.91,85.64
2015-01-07,133.51,82.01,133.89,24.50,113.27,72.47,82.09,22.52,35.32,nan,...,44.27,74.62,56.93,91.13,90.88,35.31,62.53,65.14,76.90,86.52
2015-01-08,136.71,83.17,133.41,25.44,115.28,73.21,83.97,22.70,35.74,nan,...,45.29,75.48,58.59,93.03,95.22,36.06,63.37,65.95,78.52,87.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-22,176.12,135.95,231.47,126.00,212.88,218.06,98.39,45.43,50.63,61.79,...,136.39,126.58,240.95,147.87,327.64,56.45,208.32,48.86,137.69,191.76
2021-02-23,176.17,136.94,232.46,125.86,212.12,220.18,99.63,45.51,50.54,61.44,...,135.85,127.52,235.64,149.33,329.51,57.03,212.11,48.41,135.47,197.09
2021-02-24,177.63,139.89,229.99,125.35,229.34,222.47,103.31,45.74,50.71,62.99,...,135.37,127.66,240.47,150.96,332.21,57.12,219.43,49.17,133.21,197.51


In [16]:
# sample symbol
symbol = 'MMM'

In [17]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

20.82560581139139

In [18]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [19]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.2091851963704639

In [20]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.16619884930980108

In [21]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   135.60
2015-02-28   141.78
2015-03-31   138.67
2015-04-30   131.48
2015-05-31   134.58
2015-06-30   130.54
2015-07-31   128.04
2015-08-31   121.09
2015-09-30   120.77
2015-10-31   133.93
2015-11-30   134.27
2015-12-31   129.17
2016-01-31   129.48
2016-02-29   135.49
2016-03-31   143.92
2016-04-30   144.56
2016-05-31   146.34
2016-06-30   152.26
2016-07-31   155.07
2016-08-31   156.81
2016-09-30   154.18
2016-10-31   144.61
2016-11-30   151.21
2016-12-31   157.23
2017-01-31   153.92
2017-02-28   165.14
2017-03-31   169.56
2017-04-30   173.55
2017-05-31   182.28
2017-06-30   185.60
2017-07-31   179.34
2017-08-31   183.20
2017-09-30   188.20
2017-10-31   206.39
2017-11-30   219.11
2017-12-31   212.10
2018-01-31   225.74
2018-02-28   213.48
2018-03-31   198.99
2018-04-30   176.21
2018-05-31   180.00
2018-06-30   179.54
2018-07-31   193.78
2018-08-31   193.79
2018-09-30   193.60
2018-10-31   174.81
2018-11-30   192.34
2018-12-31   176.26
2019-01-31   185.29
2019-02-28   19

In [22]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2020-10-31   -0.00
2020-11-30    0.09
2020-12-31    0.01
2021-01-31    0.00
2021-02-28    0.00
Freq: M, Name: MMM, Length: 74, dtype: float64

In [23]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.22568633296825213

In [24]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [25]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [26]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')